<a href="https://colab.research.google.com/github/bhonsleaditya1/Lending-Club-PySpark/blob/master/Lending_Club_Neural_Nets_1vR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing & Importing Libraries

In [ ]:
!pip install h5py
import gc
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import roc_auc_score,confusion_matrix,classification_report,roc_curve,auc
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils,to_categorical

#Loading Data

In [ ]:
pdf = pd.read_csv('/content/drive/My Drive/Lending-Club/loanFinal.csv', header=0, escapechar='\\')
dropcol = pd.read_csv('/content/drive/My Drive/Lending-Club/FinalDrop.csv',header=None)[0].to_list()
dbindex = pd.read_csv('/content/drive/My Drive/Lending-Club/DBRemove.csv')
pdf = pdf[pdf.amnt_left_per<=100]
#pdf = pdf[pdf.amnt_left_per>=0]
pdf = pdf.drop(dropcol,axis=1)
#pdf=pdf[pdf['application_type'].isin(['Joint App','Individual'])]
#pdf=pdf[pdf['initial_list_status'].isin(['w','f'])]
pdf = pdf.drop(dbindex['index'].to_list())
pdf = pdf.reset_index().drop('index',axis=1)
pdf.count()
pdf['year'] = pd.DatetimeIndex(pdf['issue_d']).year
#pdf = pdf.drop(dt,axis=1)
reg = pdf.select_dtypes(['float64']).columns
clas = pdf.select_dtypes(['O']).columns
gc.collect()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (41,49,117,118,119,122,123,124,127) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


22

In [ ]:
del pdf
gc.collect()

11

#Data Pre-Processing

##OneHotEncoding

In [ ]:
#OneHotEncoding 1 vs R 
pdf.loc[pdf['target']==2,'target']=0
p = pd.DataFrame(index=None)
for i in clas:
  if pdf[i].nunique()<25:
    l = pd.get_dummies(pdf[i], prefix=i)
    for j in l.columns:
      pdf[j] = l[j]
for i in clas:
  pdf = pdf.drop(i,axis=1)
pdf = pdf.rename(columns={'emp_length_< 1 year':'emp_length_less 1 year'})
timetest = pdf[pdf.year==2016]
train = pdf[pdf.year!=2016]
timetest = timetest.drop(['year'],axis=1)
train = train.drop(['year'],axis=1)
y_timetest=pd.DataFrame()
y_t = pd.DataFrame()
y_timetest['target'] = timetest['target']
y_t['target'] = train['target']
timetest = timetest.drop('target',axis=1)
train = train.drop('target',axis=1)
X_train, X_test, y_train, y_test = train_test_split(train, y_t, random_state=1301, test_size=0.3)
scale = StandardScaler()
imp = SimpleImputer(strategy="mean")
scale.fit(imp.fit_transform(X_train))
X_train = scale.transform(imp.fit_transform(X_train))
X_test = scale.transform(imp.fit_transform(X_test))
timetest = scale.transform(imp.fit_transform(timetest))
gc.collect()

0

#From this Cell
1.   1 is Class 1
2.   0 is Rest



#Base Classifiers

##MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(128, 128), max_iter=150,activation='relu',solver='adam',learning_rate='adaptive',verbose=True)
mlp.fit(X_train, y_train.values.ravel())

###Saving Model

In [ ]:
import joblib
joblib.dump(mlp, '/content/drive/My Drive/Lending-Club/Models/NN_1_mlp.pkl')

['/content/drive/My Drive/Lending-Club/Models/NN_1_mlp.pkl']

##Optimized Keras

In [ ]:
import numpy as np
np.random.seed()
import pandas as pd
from datetime import datetime
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from keras.wrappers.scikit_learn import KerasClassifier

class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

folds = 4
runs = 2

cv_LL = 0
cv_AUC = 0
cv_gini = 0
fpred = []
avpred = []
avreal = []
avids = []
patience = 10
batchsize = 2048

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' %
              (thour, tmin, round(tsec, 2)))

In [ ]:
import keras
# Let's split the data into folds. I always use the same random number for reproducibility, 
# and suggest that you do the same (you certainly don't have to use 1001).

#skf = StratifiedKFold(n_splits=folds, random_state=1001)
#starttime = timer(None)
#for i, (train_index, test_index) in enumerate(skf.split(X_train,y_train)):
#    start_time = timer(None)
#    X1, X_val1 = X_train[train_index], X_train[test_index]
#    y1, y_val1 = y_train[train_index], y_train[test_index]
    #train_ids, val_ids = tr_ids[train_index], tr_ids[test_index]
    
# This is where we define and compile the model. These parameters are not optimal, as they were chosen 
# to get a notebook to complete in 60 minutes. Other than leaving BatchNormalization and last sigmoid 
# activation alone, virtually everything else can be optimized: number of neurons, types of initializers, 
# activation functions, dropout values. The same goes for the optimizer at the end.

#########
# Never move this model definition to the beginning of the file or anywhere else outside of this loop. 
# The model needs to be initialized anew every time you run a different fold. If not, it will continue 
# the training from a previous model, and that is not what you want.
#########

    # This definition must be within the for loop or else it will continue training previous model
def baseline_model():
    model = Sequential()
    model.add(
        Dense(
            256,
            input_dim=X_train.shape[1],
            kernel_initializer='glorot_normal',
            ))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, kernel_initializer='glorot_normal'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    #model.add(Dense(50, kernel_initializer='glorot_normal'))
    #model.add(Activation('relu'))
    #model.add(BatchNormalization())
    #model.add(Dropout(0.15))
    #model.add(Dense(25, kernel_initializer='glorot_normal'))
    #model.add(Activation('relu'))
    #model.add(BatchNormalization())
    #model.add(Dropout(0.1))
    model.add(Dense(2, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adagrad', metrics = [keras.metrics.AUC()], loss='binary_crossentropy')

    return model

# This is where we repeat the runs for each fold. If you choose runs=1 above, it will run a 
# regular N-fold procedure.

#########
# It is important to leave the call to random seed here, so each run starts with a different seed.
#########

#for run in range(runs):
#  print('\n Fold %d - Run %d\n' % ((i + 1),(run + 1)))
#  np.random.seed()

# Lots to unpack here.

# The first callback prints out roc_auc and gini values at the end of each epoch. It must be listed 
# before the EarlyStopping callback, which monitors gini values saved in the previous callback. Make 
# sure to set the mode to "max" because the default value ("auto") will not handle gini properly 
# (it will act as if the model is not improving even when roc/gini go up).

# CSVLogger creates a record of all iterations. Not really needed but it doesn't hurt to have it.

# ModelCheckpoint saves a model each time gini improves. Its mode also must be set to "max" for reasons 
# explained above.

callbacks = [
    roc_auc_callback(training_data=(X_train, to_categorical(y_train)),validation_data=(X_test, to_categorical(y_test))),  # call this before EarlyStopping
    EarlyStopping(monitor='norm_gini_val', patience=patience, mode='max', verbose=1),
    CSVLogger('keras-5fold-run-01-v1-epochs.log', separator=',', append=False),
    ModelCheckpoint(
            'keras-5fold-run-01-v1-fold-' + str('%02d' % (0 + 1)) + '-run-' + str('%02d' % (0 + 1)) + '1vR.check',
            monitor='norm_gini_val', mode='max', # mode must be set to max or Keras will be confused
            save_best_only=True,
            verbose=1)
]

# The classifier is defined here. Epochs should be be set to a very large number (not 3 like below) which 
# will never be reached anyway because of early stopping. I usually put 5000 there. Because why not.

nnet = KerasClassifier(
    build_fn=baseline_model,
# Epoch needs to be set to a very large number ; early stopping will prevent it from reaching
#            epochs=5000,
    epochs=500,
    batch_size=batchsize,
    validation_data=(X_test, to_categorical(y_test)),
    verbose=2,
    shuffle=True,
    callbacks=callbacks)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train.target)

fit = nnet.fit(X_train,to_categorical(y_train),class_weight=dict(zip(np.unique(y_train),class_weights)))
        
# We want the best saved model - not the last one where the training stopped. So we delete the old 
# model instance and load the model from the last saved checkpoint. Next we predict values both for 
# validation and test data, and create a summary of parameters for each run.

del nnet
nnet = load_model('keras-5fold-run-01-v1-fold-' + str('%02d' % (0 + 1)) + '-run-' + str('%02d' % (0 + 1)) + '1vR.check')
scores_val_run = nnet.predict_proba(X_test, verbose=0)
LL_run = log_loss(X_test, scores_val_run)
print('\n Fold %d Run %d Log-loss: %.5f' % ((i + 1), (run + 1), LL_run))
AUC_run = roc_auc_score(to_categorical(y_test), scores_val_run)
print(' Fold %d Run %d AUC: %.5f' % ((i + 1), (run + 1), AUC_run))
print(' Fold %d Run %d normalized gini: %.5f' % ((i + 1), (run + 1), AUC_run*2-1))
y_pred_run = nnet.predict_proba(timetest, verbose=0)
if run > 0:
    scores_val = scores_val + scores_val_run
    y_pred = y_pred + y_pred_run
else:
    scores_val = scores_val_run
    y_pred = y_pred_run
            
# We average all runs from the same fold and provide a parameter summary for each fold. Unless something 
# is wrong, the numbers printed here should be better than any of the individual runs.

scores_val = scores_val / runs
y_pred = y_pred / runs
LL = log_loss(y_val, scores_val)
print('\n Fold %d Log-loss: %.5f' % ((i + 1), LL))
AUC = roc_auc_score(y_val, scores_val)
print(' Fold %d AUC: %.5f' % ((i + 1), AUC))
print(' Fold %d normalized gini: %.5f' % ((i + 1), AUC*2-1))
timer(start_time)

# We add up predictions on the test data for each fold. Create out-of-fold predictions for validation data.

if i > 0:
    fpred = pred + y_pred
    avreal = np.concatenate((avreal, y_val), axis=0)
    avpred = np.concatenate((avpred, scores_val), axis=0)
    avids = np.concatenate((avids, val_ids), axis=0)
else:
    fpred = y_pred
    avreal = y_val
    avpred = scores_val
    avids = val_ids
pred = fpred
cv_LL = cv_LL + LL
cv_AUC = cv_AUC + AUC
cv_gini = cv_gini + (AUC*2-1)

###Metrics for best model

In [ ]:
model = Sequential()
model.add(
    Dense(
        256,
        input_dim=X_train.shape[1],
        kernel_initializer='glorot_normal',
        ))
model.add(Activation('tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer='glorot_normal'))
model.add(Activation('tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(2, activation='sigmoid'))
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-6,decay_steps=10000,decay_rate=0.9)
#sgd = keras.optimizers.Adagrad(learning_rate=lr_schedule)

model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=[keras.metrics.AUC()])
model.load_weights('keras-5fold-run-01-v1-fold-' + str('%02d' % (0 + 1)) + '-run-' + str('%02d' % (0 + 1)) + '1vR.check')
nnet=model
#model.load_weights('keras-5fold-run-01-v1-fold-' + str('%02d' % (i + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check')
scores_val_run = nnet.predict_proba(X_test, verbose=0)
LL_run = log_loss(X_test, scores_val_run)
print('\n Fold %d Run %d Log-loss: %.5f' % ((i + 1), (run + 1), LL_run))
AUC_run = roc_auc_score(to_categorical(y_test), scores_val_run)
print(' Fold %d Run %d AUC: %.5f' % ((i + 1), (run + 1), AUC_run))
print(' Fold %d Run %d normalized gini: %.5f' % ((i + 1), (run + 1), AUC_run*2-1))
y_pred_run = nnet.predict_proba(timetest, verbose=0)
if run > 0:
    scores_val = scores_val + scores_val_run
    y_pred = y_pred + y_pred_run
else:
    scores_val = scores_val_run
    y_pred = y_pred_run
            
# We average all runs from the same fold and provide a parameter summary for each fold. Unless something 
# is wrong, the numbers printed here should be better than any of the individual runs.

scores_val = scores_val / runs
y_pred = y_pred / runs
LL = log_loss(y_val, scores_val)
print('\n Fold %d Log-loss: %.5f' % ((i + 1), LL))
AUC = roc_auc_score(y_val, scores_val)
print(' Fold %d AUC: %.5f' % ((i + 1), AUC))
print(' Fold %d normalized gini: %.5f' % ((i + 1), AUC*2-1))
timer(start_time)

# We add up predictions on the test data for each fold. Create out-of-fold predictions for validation data.

if i > 0:
    fpred = pred + y_pred
    avreal = np.concatenate((avreal, y_val), axis=0)
    avpred = np.concatenate((avpred, scores_val), axis=0)
    avids = np.concatenate((avids, val_ids), axis=0)
else:
    fpred = y_pred
    avreal = y_val
    avpred = scores_val
    avids = val_ids
pred = fpred
cv_LL = cv_LL + LL
cv_AUC = cv_AUC + AUC
cv_gini = cv_gini + (AUC*2-1)

##SGD

In [ ]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils,to_categorical
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train.target)
class_weights=dict(zip(np.unique(y_train),class_weights))
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-6,
    decay_steps=10000,
    decay_rate=0.9)
sgd = keras.optimizers.SGD(learning_rate=lr_schedule)

model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=[keras.metrics.AUC()])
model.fit(X_train, np_utils.to_categorical(y_train),validation_data=(X_test,to_categorical(y_test)), epochs=200, batch_size=1024,use_multiprocessing=True,shuffle=True,class_weight=class_weights)
#Saving model
model.save_weights("/content/drive/My Drive/Lending-Club/Models/NN_1_SGD_model.h5")

##SGD Nesterov

In [ ]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils,to_categorical
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train.target)
class_weights=dict(zip(np.unique(y_train),class_weights))
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-6,
    decay_steps=10000,
    decay_rate=0.9)
sgd = keras.optimizers.SGD(learning_rate=lr_schedule,nesterov=True)

model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=[keras.metrics.AUC()])
model.fit(X_train, np_utils.to_categorical(y_train),validation_data=(X_test,to_categorical(y_test)), epochs=200, batch_size=1024,use_multiprocessing=True,shuffle=True,class_weight=class_weights)
#Saving model
model.save_weights("/content/drive/My Drive/Lending-Club/Models/NN_1_SGD_N_model.h5")

##Adam

In [ ]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils,to_categorical
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train.target)
class_weights=dict(zip(np.unique(y_train),class_weights))
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-6,
    decay_steps=10000,
    decay_rate=0.9)
sgd = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=[keras.metrics.AUC()])
model.fit(X_train, np_utils.to_categorical(y_train),validation_data=(X_test,to_categorical(y_test)), epochs=200, batch_size=1024,use_multiprocessing=True,shuffle=True,class_weight=class_weights)
#Saving model
model.save_weights("/content/drive/My Drive/Lending-Club/Models/NN_1_Adam_model.h5")

##Keras Bayesian Optimization

###Trial Space

In [ ]:
!pip install -q keras-tuner
from kerastuner import HyperModel
from tensorflow.keras import layers
import tensorflow
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import utils
from keras.utils import np_utils,to_categorical

class ClassificationHyperModel(HyperModel):
    def __init__(self, input_shape):
      self.input_shape = input_shape

    def build(self, hp):
        model = tensorflow.keras.Sequential()

        model.add(
            layers.Dense(
                units=hp.Choice('units', values=[64,128,256,512], default=128),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=input_shape
            )
        )
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.2,
                    max_value=0.5,
                    default=0.25,
                    step=0.1)
            )
        )
        
        model.add(
            layers.Dense(
                units=hp.Choice('units', values=[64,128,256,512], default=128),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu')
            )
        )
        
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.2,
                    max_value=0.5,
                    default=0.25,
                    step=0.1)
            )
        )
        
        model.add(layers.Dense(2,activation='softmax'))
        
        model.compile(
            hp.Choice('optimizer',values=['Adagrad','Sgd','Adam'],default='Adam')
            ,loss='binary_crossentropy',metrics=[keras.metrics.AUC()]
        )
        
        return model

     |████████████████████████████████| 61kB 2.5MB/s 


Using TensorFlow backend.


###Run Trials

In [ ]:
import kerastuner
from tensorflow import keras
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train.target)
class_weights=dict(zip(np.unique(y_train),class_weights))
input_shape = (X_train.shape[1],)
hypermodel = ClassificationHyperModel(input_shape)
tuner_bo = kerastuner.tuners.BayesianOptimization(
            hypermodel,
            objective=kerastuner.Objective("auc", direction="max"),
            max_trials=2,
            seed=42,
            executions_per_trial=2
        )
tuner_bo.search(X_train, to_categorical(y_train), epochs=100, validation_split=0.3, verbose=0,class_weight=class_weights)
best_model = tuner_bo.get_best_models(num_models=1)[0]
best_model.evaluate(X_test, to_categorical(y_test))
gc.collect()

INFO:tensorflow:Oracle triggered exit
9827/9827 [==============================] - 30s 3ms/step - loss: 0.5034 - auc: 0.8345


715

###Saving Trial Results

In [ ]:
!zip -r /content/1vR_keras.tuner.BayesOpt.zip /content/untitled_project

  adding: content/untitled_project/ (stored 0%)
  adding: content/untitled_project/oracle.json (deflated 55%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/ (stored 0%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/trial.json (deflated 62%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/checkpoints/ (stored 0%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/checkpoints/epoch_0/ (stored 0%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/checkpoints/epoch_0/checkpoint.data-00001-of-00002 (deflated 8%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/checkpoints/epoch_0/checkpoint.index (deflated 62%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/checkpoints/epoch_0/checkpoint (deflated 48%)
  adding: content/untitled_project/trial_8f7491d1ba3cc5789a6e3291a8bf0b24/checkpoints/epoch_0/checkpoint.data-00

#Analysing Model

##Input Model

In [ ]:
model = #input the model you want to analyse

##Percentile f1 score

In [ ]:
from sklearn.metrics import f1_score
res = model.predict_proba(X_train)
th1=1-(y_train.target==0).sum()/y_train.target.count()*1
th2=1-(y_train.target==1).sum()/y_train.target.count()*1
c1=pd.DataFrame(res[:,0])
c2=pd.DataFrame(res[:,1])
print(f1_score(np.where(c1>c1.quantile(th1),1,0),np.where(y_train==0,1,0)))
print(f1_score(np.where(c2>c2.quantile(th2),1,0),np.where(y_train==1,1,0)))
res = model.predict_proba(X_test)
th1=1-(y_test.target==0).sum()/y_test.target.count()*1
th2=1-(y_test.target==1).sum()/y_test.target.count()*1
c1=pd.DataFrame(res[:,0])
c2=pd.DataFrame(res[:,1])
print(f1_score(np.where(c1>c1.quantile(th1),1,0),np.where(y_test==0,1,0)))
print(f1_score(np.where(c2>c2.quantile(th2),1,0),np.where(y_test==1,1,0)))
res = model.predict_proba(timetest)
th1=1-(y_timetest.target==0).sum()/y_timetest.target.count()*1
th2=1-(y_timetest.target==1).sum()/y_timetest.target.count()*1
c1=pd.DataFrame(res[:,0])
c2=pd.DataFrame(res[:,1])
print(f1_score(np.where(c1>c1.quantile(th1),1,0),np.where(y_timetest==0,1,0)))
print(f1_score(np.where(c2>c2.quantile(th2),1,0),np.where(y_timetest==1,1,0)))

##Gini per Class

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
gini =0
#No of classes is j
j=2
y_predtrain=model.predict_proba(X_train)
y_pred=model.predict_proba(X_test)
y_predtime=model.predict_proba(timetest)
drop_enc = OneHotEncoder().fit(pd.DataFrame(y_test))
y_gini = drop_enc.transform(pd.DataFrame(y_test)).toarray()
drop_enc = OneHotEncoder().fit(pd.DataFrame(y_timetest))
y_ginitime = drop_enc.transform(pd.DataFrame(y_timetest)).toarray()
drop_enc = OneHotEncoder().fit(pd.DataFrame(y_train))
y_ginitrain = drop_enc.transform(pd.DataFrame(y_train)).toarray()
for i in range(j):
    fpr[i], tpr[i], _ = roc_curve(y_ginitrain[:, i], y_predtrain[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(2*roc_auc[i]-1)
for i in range(j):
    fpr[i], tpr[i], _ = roc_curve(y_gini[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(2*roc_auc[i]-1)
for i in range(j):
    fpr[i], tpr[i], _ = roc_curve(y_ginitime[:, i], y_predtime[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(2*roc_auc[i]-1)

##Accuracy

In [ ]:
print(accuracy_score(y_train,model.predict(X_train)))
print(accuracy_score(y_test,model.predict(X_test)))
print(accuracy_score(y_timetest,model.predict(timetest)))

##Classification Report

In [ ]:
y_pred = model.predict_classes(X_test)
y_predtrain = model.predict_classes(X_train)
y_predtime = model.predict_classes(timetest)
print(confusion_matrix(y_train,y_predtrain))
print(classification_report(y_train,y_predtrain))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_timetest,y_predtime))
print(classification_report(y_timetest,y_predtime))

#Feature Importance

##Input Model

In [ ]:
model_f = #input the model you want to find feature Importance
model_name = #input model name

##Feature Importance

In [ ]:
from sklearn.inspection import permutation_importance
perm = permutation_importance(model_f,X_train,y_train)

In [ ]:
r = perm
feature ={}
for i in r.importances_mean.argsort()[::-1]:
  if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
    feature[train.columns[i]]={'mean':r.importances_mean[i],'std':r.importances_std[i]}

In [ ]:
pd.DataFrame.from_dict(feature).T.to_excel('1vR_feature_neural.xlsx')